In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/covertype-target/target.csv
/kaggle/input/forest-cover-type-kernels-only/test.csv
/kaggle/input/forest-cover-type-kernels-only/train.csv
/kaggle/input/forest-cover-type-kernels-only/sample_submission.csv


In [2]:
%%time
train_df = pd.read_csv('/kaggle/input/forest-cover-type-kernels-only/train.csv')
test_df = pd.read_csv('/kaggle/input/forest-cover-type-kernels-only/test.csv')
submission_df = pd.read_csv('/kaggle/input/forest-cover-type-kernels-only/sample_submission.csv')

CPU times: user 1.76 s, sys: 724 ms, total: 2.48 s
Wall time: 2.33 s


In [3]:
train_df.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,...,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,...,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,...,0,0,0,0,0,0,0,0,0,5


In [4]:
test_df['Cover_Type'] = np.nan
data_df = pd.concat([train_df, test_df]).drop(columns=['Cover_Type'])

In [5]:
print(train_df.shape)
print(test_df.shape)
print(data_df.shape)

(15120, 56)
(565892, 56)
(581012, 55)


# Read Target Dataset

In [6]:
target_all = pd.read_csv('/kaggle/input/covertype-target/target.csv',
                        index_col='Id')

y_test = target_all.loc[test_df['Id']]

# Convert One-Hot encoding to label

In [7]:

cat_soil = [col for col in data_df.columns if 'Soil' in col]
data_df['Soil_Type'] = data_df[cat_soil].values.argmax(axis=1)+1
data_df = data_df.drop(columns=cat_soil)


cat_wilderness = [col for col in data_df.columns if 'Wilderness' in col]
data_df['Wilderness_Area'] = data_df[cat_wilderness].values.argmax(axis=1)+1
data_df = data_df.drop(columns=cat_wilderness)

In [8]:
data_df.shape

(581012, 13)

In [9]:
data_df.columns

Index(['Id', 'Elevation', 'Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Soil_Type',
       'Wilderness_Area'],
      dtype='object')

In [10]:
train_df.columns

Index(['Id', 'Elevation', 'Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points',
       'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3',
       'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3',
       'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8',
       'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12',
       'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16',
       'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20',
       'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24',
       'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28',
       'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32',
       'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36',
       'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_

In [11]:
import featuretools as ft
import featuretools.variable_types as vtypes
from multiprocessing import cpu_count

In [12]:
from IPython.display import display

for i in range(0, len(ft.list_primitives()), 10):
#     print(i)
    
    display(ft.list_primitives()[0+i:i+10])

,name,type,description
0,min,aggregation,"Calculates the smallest value, ignoring `NaN` ..."
1,skew,aggregation,Computes the extent to which a distribution di...
2,time_since_first,aggregation,Calculates the time elapsed since the first da...
3,avg_time_between,aggregation,Computes the average number of seconds between...
4,median,aggregation,Determines the middlemost number in a list of ...
5,mode,aggregation,Determines the most commonly repeated value.
6,n_most_common,aggregation,Determines the `n` most common elements.
7,time_since_last,aggregation,Calculates the time elapsed since the last dat...
8,mean,aggregation,Computes the average for a list of values.
9,percent_true,aggregation,Determines the percent of `True` values.


,name,type,description
10,all,aggregation,Calculates if all values are 'True' in a list.
11,trend,aggregation,Calculates the trend of a variable over time.
12,sum,aggregation,"Calculates the total addition, ignoring `NaN`."
13,num_true,aggregation,Counts the number of `True` values.
14,any,aggregation,Determines if any value is 'True' in a list.
15,count,aggregation,"Determines the total number of values, excludi..."
16,num_unique,aggregation,"Determines the number of distinct values, igno..."
17,last,aggregation,Determines the last value in a list.
18,max,aggregation,"Calculates the highest value, ignoring `NaN` v..."
19,std,aggregation,Computes the dispersion relative to the mean v...


,name,type,description
20,cum_min,transform,Calculates the cumulative minimum.
21,not,transform,Negates a boolean value.
22,hour,transform,Determines the hour value of a datetime.
23,cum_count,transform,Calculates the cumulative count.
24,equal_scalar,transform,Determines if values in a list are equal to a ...
25,add_numeric_scalar,transform,Add a scalar to each value in the list.
26,diff,transform,Compute the difference between the value in a ...
27,is_weekend,transform,Determines if a date falls on a weekend.
28,num_words,transform,Determines the number of words in a string by ...
29,divide_numeric,transform,Element-wise division of two lists.


,name,type,description
30,less_than_equal_to_scalar,transform,Determines if values are less than or equal to...
31,month,transform,Determines the month value of a datetime.
32,not_equal_scalar,transform,Determines if values in a list are not equal t...
33,or,transform,Element-wise logical OR of two lists.
34,longitude,transform,Returns the second tuple value in a list of La...
35,absolute,transform,Computes the absolute value of a number.
36,less_than_scalar,transform,Determines if values are less than a given sca...
37,greater_than_equal_to_scalar,transform,Determines if values are greater than or equal...
38,multiply_numeric,transform,Element-wise multiplication of two lists.
39,greater_than_scalar,transform,Determines if values are greater than a given ...


,name,type,description
40,modulo_by_feature,transform,Return the modulo of a scalar by each element ...
41,subtract_numeric_scalar,transform,Subtract a scalar from each element in the list.
42,divide_by_feature,transform,Divide a scalar by each value in the list.
43,modulo_numeric,transform,Element-wise modulo of two lists.
44,week,transform,Determines the week of the year from a datetime.
45,cum_max,transform,Calculates the cumulative maximum.
46,percentile,transform,Determines the percentile rank for each value ...
47,haversine,transform,Calculates the approximate haversine distance ...
48,second,transform,Determines the seconds value of a datetime.
49,cum_mean,transform,Calculates the cumulative mean.


,name,type,description
50,negate,transform,Negates a numeric value.
51,subtract_numeric,transform,Element-wise subtraction of two lists.
52,time_since,transform,Calculates time from a value to a specified cu...
53,day,transform,Determines the day of the month from a datetime.
54,cum_sum,transform,Calculates the cumulative sum.
55,equal,transform,Determines if values in one list are equal to ...
56,add_numeric,transform,Element-wise addition of two lists.
57,isin,transform,Determines whether a value is present in a pro...
58,num_characters,transform,Calculates the number of characters in a string.
59,year,transform,Determines the year value of a datetime.


,name,type,description
60,time_since_previous,transform,Compute the time since the previous entry in a...
61,less_than,transform,Determines if values in one list are less than...
62,less_than_equal_to,transform,Determines if values in one list are less than...
63,multiply_numeric_scalar,transform,Multiply each element in the list by a scalar.
64,not_equal,transform,Determines if values in one list are not equal...
65,is_null,transform,Determines if a value is null.
66,and,transform,Element-wise logical AND of two lists.
67,greater_than_equal_to,transform,Determines if values in one list are greater t...
68,scalar_subtract_numeric_feature,transform,Subtract each value in the list from a given s...
69,weekday,transform,Determines the day of the week from a datetime.


,name,type,description
70,greater_than,transform,Determines if values in one list are greater t...
71,modulo_numeric_scalar,transform,Return the modulo of each element in the list ...
72,divide_numeric_scalar,transform,Divide each element in the list by a scalar.
73,latitude,transform,Returns the first tuple value in a list of Lat...
74,minute,transform,Determines the minutes value of a datetime.


# Feature Engineering

Add `normalize_entity`

In [13]:
variable_type = {col: vtypes.Categorical for col in data_df.columns if 'Soil' in col or 'Wilderness' in col}

# variable_type['Cover_Type'] = vtypes.Categorical

print(variable_type)

{'Soil_Type': <class 'featuretools.variable_types.variable.Categorical'>, 'Wilderness_Area': <class 'featuretools.variable_types.variable.Categorical'>}


In [14]:
es = ft.EntitySet('Dataset')

es.entity_from_dataframe(entity_id='forest_table',
                         index='Id',
                        dataframe=data_df,
                         variable_types=variable_type
                        )

# es.normalize_entity(base_entity_id='forest_table',
#                    new_entity_id='soil_type_table',
#                    index='Soil_Type',
# #                    additional_variables=['Cover_Type'],
#                    )

# es.normalize_entity(base_entity_id='forest_table',
#                    new_entity_id='wilderness_area_table',
#                    index='Wilderness_Area',
#                    additional_variables=['Elevation', 'Slope'])

Entityset: Dataset
  Entities:
    forest_table [Rows: 581012, Columns: 13]
  Relationships:
    No relationships

In [15]:
es['forest_table'].variables

[<Variable: Id (dtype = index)>,
 <Variable: Elevation (dtype = numeric)>,
 <Variable: Aspect (dtype = numeric)>,
 <Variable: Slope (dtype = numeric)>,
 <Variable: Horizontal_Distance_To_Hydrology (dtype = numeric)>,
 <Variable: Vertical_Distance_To_Hydrology (dtype = numeric)>,
 <Variable: Horizontal_Distance_To_Roadways (dtype = numeric)>,
 <Variable: Hillshade_9am (dtype = numeric)>,
 <Variable: Hillshade_Noon (dtype = numeric)>,
 <Variable: Hillshade_3pm (dtype = numeric)>,
 <Variable: Horizontal_Distance_To_Fire_Points (dtype = numeric)>,
 <Variable: Soil_Type (dtype = categorical)>,
 <Variable: Wilderness_Area (dtype = categorical)>]

In [16]:
es['forest_table'].df

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Soil_Type,Wilderness_Area
1,1,2596,51,3,258,0,510,221,232,148,6279,29,1
2,2,2590,56,2,212,-6,390,220,235,151,6225,29,1
3,3,2804,139,9,268,65,3180,234,238,135,6121,12,1
4,4,2785,155,18,242,118,3090,238,238,122,6211,30,1
5,5,2595,45,2,153,-1,391,220,234,150,6172,29,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581008,581008,2396,153,20,85,17,108,240,237,118,837,2,3
581009,581009,2391,152,19,67,12,95,240,237,119,845,2,3
581010,581010,2386,159,17,60,7,90,236,241,130,854,2,3
581011,581011,2384,170,15,60,5,90,230,245,143,864,2,3


In [17]:
from featuretools.primitives import make_trans_primitive
from featuretools.variable_types import Numeric

def squared_root(col1, col2):
    return (col1**2 + col2**2)**0.5

Squared_Root = make_trans_primitive(function=squared_root,
                                   input_types=[Numeric, Numeric],
                                   return_type=Numeric,
                                    commutative=True, # True: combination, False: permutaion
                                   )


def abs_diff(col1, col2):
    return abs(col1 - col2)

Abs_Diff = make_trans_primitive(function=abs_diff,
                               input_types=[Numeric, Numeric],
                               return_type=Numeric,
                               commutative=True, # True: combination, False: permutaion
                               )


def mean_2_cols(col1, col2):
    return (col1.add(col2)) / 2


Mean_2_Cols = make_trans_primitive(function=mean_2_cols,
                                  input_types=[Numeric, Numeric],
                                  return_type=Numeric,
                                  commutative=True, # True: combination, False: permutaion
                                  )


def mean_3_cols(col1, col2, col3):
    return (col1.add(col2.add(col3))) / 3

Mean_3_Cols = make_trans_primitive(function=mean_3_cols,
                                  input_types=[Numeric, Numeric, Numeric],
                                  return_type=Numeric,
                                  commutative=True, # True: combination, False: permutaion
                                  )


def div_2_cols(col1, col2):
    eps = 10e-2
    return (col1+eps) / (col2+eps)


Div_2_Cols = make_trans_primitive(function=div_2_cols,
                                  input_types=[Numeric, Numeric],
                                  return_type=Numeric,
                                  commutative=True, # True: combination, False: permutaion
                                  )

In [18]:
%%time
feature_matrix, features = ft.dfs(entityset=es,
                                 target_entity='forest_table',
                                  trans_primitives=[
#                                                     Squared_Root,
#                                                     Abs_Diff,
                                                    Mean_3_Cols,
#                                                       Div_2_Cols,
#                                                     'add_numeric', 
#                                                   'subtract_numeric',
#                                                     'multiply_numeric',
#                                                     'divide_numeric',  # divide by 0.
#                                                     'subtract_numeric',
                                                   ],
                                 max_depth=1,
                                 n_jobs=cpu_count())

EntitySet scattered to 4 workers in 10 seconds
CPU times: user 4.78 s, sys: 3.03 s, total: 7.8 s
Wall time: 24.5 s


In [19]:
feature_matrix = feature_matrix.reset_index()

In [20]:
feature_matrix.shape

(581012, 133)

In [21]:
data_df.shape

(581012, 13)

In [22]:
features

[<Feature: Elevation>,
 <Feature: Aspect>,
 <Feature: Slope>,
 <Feature: Horizontal_Distance_To_Hydrology>,
 <Feature: Vertical_Distance_To_Hydrology>,
 <Feature: Horizontal_Distance_To_Roadways>,
 <Feature: Hillshade_9am>,
 <Feature: Hillshade_Noon>,
 <Feature: Hillshade_3pm>,
 <Feature: Horizontal_Distance_To_Fire_Points>,
 <Feature: Soil_Type>,
 <Feature: Wilderness_Area>,
 <Feature: MEAN_3_COLS(Elevation, Hillshade_9am, Vertical_Distance_To_Hydrology)>,
 <Feature: MEAN_3_COLS(Hillshade_3pm, Hillshade_Noon, Horizontal_Distance_To_Fire_Points)>,
 <Feature: MEAN_3_COLS(Hillshade_9am, Horizontal_Distance_To_Fire_Points, Vertical_Distance_To_Hydrology)>,
 <Feature: MEAN_3_COLS(Aspect, Hillshade_3pm, Hillshade_9am)>,
 <Feature: MEAN_3_COLS(Aspect, Elevation, Hillshade_3pm)>,
 <Feature: MEAN_3_COLS(Hillshade_3pm, Horizontal_Distance_To_Fire_Points, Vertical_Distance_To_Hydrology)>,
 <Feature: MEAN_3_COLS(Aspect, Elevation, Horizontal_Distance_To_Hydrology)>,
 <Feature: MEAN_3_COLS(Aspect,

### Check commutative of average function
It has to be combination rather than permutation.

In [23]:
for f in features:
    name = f.get_name()
    if 'Aspect' in name and 'Elevation' in name and 'Hillshade_3pm' in name:
        print(name)

MEAN_3_COLS(Aspect, Elevation, Hillshade_3pm)


# Prepare Train/Test Dataset

In [24]:
feature_matrix = feature_matrix.set_index('Id')

In [25]:
indice = train_df['Id']
# X_train = feature_matrix.loc[indice].drop(columns=['soil_type_table.Cover_Type'])
# y_train = feature_matrix.loc[indice, 'soil_type_table.Cover_Type'].astype('category')
# y_train = train_df['Cover_Type']
X_train = feature_matrix.loc[indice]
y_train = train_df['Cover_Type'].astype('category')

indice = test_df['Id']
# X_test = feature_matrix.loc[indice].drop(columns=['soil_type_table.Cover_Type'])
X_test = feature_matrix.loc[indice]

### Recycle `feature_matrix`

In [26]:
import gc

del feature_matrix
# del features

gc.collect()

676

In [27]:
from IPython.display import display

display(X_train.tail())
display(train_df.tail())

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,"MEAN_3_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways, Slope)","MEAN_3_COLS(Elevation, Horizontal_Distance_To_Hydrology, Horizontal_Distance_To_Roadways)","MEAN_3_COLS(Elevation, Hillshade_9am, Hillshade_Noon)","MEAN_3_COLS(Hillshade_Noon, Horizontal_Distance_To_Fire_Points, Slope)","MEAN_3_COLS(Aspect, Hillshade_Noon, Slope)","MEAN_3_COLS(Elevation, Horizontal_Distance_To_Fire_Points, Slope)","MEAN_3_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways, Vertical_Distance_To_Hydrology)","MEAN_3_COLS(Aspect, Horizontal_Distance_To_Hydrology, Slope)","MEAN_3_COLS(Horizontal_Distance_To_Hydrology, Horizontal_Distance_To_Roadways, Vertical_Distance_To_Hydrology)","MEAN_3_COLS(Hillshade_3pm, Hillshade_Noon, Horizontal_Distance_To_Roadways)"
Id,,,,,,,,,,,,,,,,,,,,,
15116,2607,243,23,258,7,660,170,251,214,1282,...,655.000000,1175.000000,1009.333333,518.666667,172.333333,1304.000000,649.666667,174.666667,308.333333,375.000000
15117,2603,121,19,633,195,618,249,221,91,1325,...,654.000000,1284.666667,1024.333333,521.666667,120.333333,1315.666667,712.666667,257.666667,482.000000,310.000000
15118,2492,134,25,365,117,335,250,220,83,1187,...,515.666667,1064.000000,987.333333,477.333333,126.333333,1234.666667,546.333333,174.666667,272.333333,212.666667
15119,2487,167,28,218,101,242,229,237,119,932,...,400.666667,982.333333,984.333333,399.000000,144.000000,1149.000000,425.000000,137.666667,187.000000,199.333333
15120,2475,197,34,319,78,270,189,244,164,914,...,406.000000,1021.333333,969.333333,397.333333,158.333333,1141.000000,420.666667,183.333333,222.333333,226.000000


,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
15115,15116,2607,243,23,258,7,660,170,251,214,...,0,0,0,0,0,0,0,0,0,3
15116,15117,2603,121,19,633,195,618,249,221,91,...,0,0,0,0,0,0,0,0,0,3
15117,15118,2492,134,25,365,117,335,250,220,83,...,0,0,0,0,0,0,0,0,0,3
15118,15119,2487,167,28,218,101,242,229,237,119,...,0,0,0,0,0,0,0,0,0,3
15119,15120,2475,197,34,319,78,270,189,244,164,...,0,0,0,0,0,0,0,0,0,3


In [28]:
display(X_test.tail())
display(test_df.tail())

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,"MEAN_3_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways, Slope)","MEAN_3_COLS(Elevation, Horizontal_Distance_To_Hydrology, Horizontal_Distance_To_Roadways)","MEAN_3_COLS(Elevation, Hillshade_9am, Hillshade_Noon)","MEAN_3_COLS(Hillshade_Noon, Horizontal_Distance_To_Fire_Points, Slope)","MEAN_3_COLS(Aspect, Hillshade_Noon, Slope)","MEAN_3_COLS(Elevation, Horizontal_Distance_To_Fire_Points, Slope)","MEAN_3_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways, Vertical_Distance_To_Hydrology)","MEAN_3_COLS(Aspect, Horizontal_Distance_To_Hydrology, Slope)","MEAN_3_COLS(Horizontal_Distance_To_Hydrology, Horizontal_Distance_To_Roadways, Vertical_Distance_To_Hydrology)","MEAN_3_COLS(Hillshade_3pm, Hillshade_Noon, Horizontal_Distance_To_Roadways)"
Id,,,,,,,,,,,,,,,,,,,,,
581008,2396,153,20,85,17,108,240,237,118,837,...,321.666667,863.000000,957.666667,364.666667,136.666667,1084.333333,320.666667,86.000000,70.000000,154.333333
581009,2391,152,19,67,12,95,240,237,119,845,...,319.666667,851.000000,956.000000,367.000000,136.000000,1085.000000,317.333333,79.333333,58.000000,150.333333
581010,2386,159,17,60,7,90,236,241,130,854,...,320.333333,845.333333,954.333333,370.666667,139.000000,1085.666667,317.000000,78.666667,52.333333,153.666667
581011,2384,170,15,60,5,90,230,245,143,864,...,323.000000,844.666667,953.000000,374.666667,143.333333,1087.666667,319.666667,81.666667,51.666667,159.333333
581012,2383,165,13,60,4,67,231,244,141,875,...,318.333333,836.666667,952.666667,377.333333,140.666667,1090.333333,315.333333,79.333333,43.666667,150.666667


,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
565887,581008,2396,153,20,85,17,108,240,237,118,...,0,0,0,0,0,0,0,0,0,NaN
565888,581009,2391,152,19,67,12,95,240,237,119,...,0,0,0,0,0,0,0,0,0,NaN
565889,581010,2386,159,17,60,7,90,236,241,130,...,0,0,0,0,0,0,0,0,0,NaN
565890,581011,2384,170,15,60,5,90,230,245,143,...,0,0,0,0,0,0,0,0,0,NaN
565891,581012,2383,165,13,60,4,67,231,244,141,...,0,0,0,0,0,0,0,0,0,NaN


# Training

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.metrics import accuracy_score

from tqdm import tqdm_notebook

from collections import OrderedDict

def fit_predict(model, X_train=X_train, y_train=y_train, 
                        X_test=X_test, y_test=y_test):
    
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    score = accuracy_score(predictions, y_test)
    
    return score


def fit_pipelines(pipelines, 
                  X_train=X_train, y_train=y_train,
                 X_test=X_test, y_test=y_test):
    
    scores = {}
    for name, pipeline in tqdm_notebook(pipelines.items()):
        print('Start fitting using {}'.format(name))
        pipeline.fit(X_train, y_train)

        predictions = pipeline.predict(X_test)
        score = accuracy_score(predictions, y_test)
        scores[name] = score

        print('{:20s}: score={}'.format(name, score))

    return(scores)

In [30]:
%%time


seed=47
pipelines = {}

pipelines['Random Forest'] = Pipeline(steps=[('RF', 
                                            RandomForestClassifier(n_jobs=-1,
                                                                   n_estimators=300,
#                                                                    max_features=7,
#                                                                    max_leaf_nodes=220,
                                                                   random_state=seed
                                                                   )
                                                    ),])

pipelines['Extra Trees'] = Pipeline(
                                    steps=[('random forest extra', ExtraTreesClassifier(n_estimators=100, random_state=seed))]
                                    )

pipelines['KNN'] = Pipeline(steps=[
                                        ('KNN', KNeighborsClassifier(n_neighbors=2))
                                    ])

pipelines['LGBM'] = Pipeline(steps=[('LightGBM', 
                                    lgb.LGBMClassifier(n_estimators=300,
                                                             num_class=8,
                                                            num_leaves=32,
                                                            random_state=seed,)
                                   ),])



scores = fit_pipelines(pipelines)

Start fitting using Random Forest
Random Forest       : score=0.7465859209884571
Start fitting using Extra Trees
Extra Trees         : score=0.7801294239890297
Start fitting using KNN
KNN                 : score=0.6867370452312456
Start fitting using LGBM
LGBM                : score=0.7536667774062895

CPU times: user 8min 7s, sys: 8.78 s, total: 8min 16s
Wall time: 3min 30s


# Feature Selection

In [31]:
from sklearn.feature_selection import SelectFromModel


In [32]:
features_var = pd.DataFrame(index=X_train.columns)
name_classifiers = []

for name, p in tqdm_notebook(pipelines.items()):
    
    # cannot apply feature selection to KNN
    if 'KNN'.lower() in name.lower():
        continue

    name_classifiers.append(name)
    
#     step_name = p.steps[-1][0]
#     print(step_name)
    clf = p.steps[-1][1]
#     print(clf)

    model = SelectFromModel(clf, prefit=True)
    X_train_new = model.transform(X_train)
    X_train_inversed = model.inverse_transform(X_train_new)
    selected_features = pd.DataFrame(X_train_inversed,
                                    index=X_train.index,
                                    columns=X_train.columns)
    
    features_var[name] = selected_features.var()
    

In [33]:
features_var['mean'] = features_var[name_classifiers].mean(axis=1)
features_var['var'] = features_var[name_classifiers].var(axis=1)

features_var[:20]

,Random Forest,Extra Trees,LGBM,mean,var
Elevation,174455.068186,174455.068186,174455.068186,174455.068186,0.000000e+00
Aspect,0.000000,0.000000,0.000000,0.000000,0.000000e+00
Slope,0.000000,0.000000,71.468878,23.822959,1.702600e+03
Horizontal_Distance_To_Hydrology,0.000000,0.000000,0.000000,0.000000,0.000000e+00
Vertical_Distance_To_Hydrology,0.000000,0.000000,3750.264863,1250.088288,4.688162e+06
Horizontal_Distance_To_Roadways,0.000000,0.000000,0.000000,0.000000,0.000000e+00
Hillshade_9am,933.992256,933.992256,933.992256,933.992256,0.000000e+00
Hillshade_Noon,0.000000,0.000000,519.929633,173.309878,9.010894e+04
Hillshade_3pm,0.000000,0.000000,2106.368347,702.122782,1.478929e+06
Horizontal_Distance_To_Fire_Points,0.000000,0.000000,0.000000,0.000000,0.000000e+00


In [34]:
display(pd.DataFrame(features_var['mean'].sort_values(ascending=False)))
display(pd.DataFrame(features_var['var'].sort_values(ascending=False)))

,mean
"MEAN_3_COLS(Elevation, Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)",207629.704868
Elevation,174455.068186
"MEAN_3_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Hydrology, Horizontal_Distance_To_Roadways)",170897.342783
"MEAN_3_COLS(Hillshade_3pm, Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)",163373.245490
"MEAN_3_COLS(Aspect, Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)",163046.591632
...,...
"MEAN_3_COLS(Aspect, Horizontal_Distance_To_Hydrology, Vertical_Distance_To_Hydrology)",0.000000
"MEAN_3_COLS(Aspect, Horizontal_Distance_To_Hydrology, Horizontal_Distance_To_Roadways)",0.000000
"MEAN_3_COLS(Aspect, Horizontal_Distance_To_Roadways, Slope)",0.000000
"MEAN_3_COLS(Hillshade_Noon, Horizontal_Distance_To_Roadways, Vertical_Distance_To_Hydrology)",0.000000


,var
"MEAN_3_COLS(Elevation, Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)",1.293303e+11
"MEAN_3_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Hydrology, Horizontal_Distance_To_Roadways)",8.761771e+10
"MEAN_3_COLS(Hillshade_3pm, Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)",8.007245e+10
"MEAN_3_COLS(Aspect, Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)",7.975257e+10
"MEAN_3_COLS(Hillshade_9am, Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)",7.928654e+10
...,...
"MEAN_3_COLS(Aspect, Horizontal_Distance_To_Roadways, Slope)",0.000000e+00
"MEAN_3_COLS(Hillshade_Noon, Horizontal_Distance_To_Roadways, Vertical_Distance_To_Hydrology)",0.000000e+00
"MEAN_3_COLS(Aspect, Hillshade_3pm, Horizontal_Distance_To_Fire_Points)",0.000000e+00
"MEAN_3_COLS(Elevation, Hillshade_9am, Horizontal_Distance_To_Fire_Points)",0.000000e+00


### Show Top-10 mean and variation

In [35]:
original_features = list(data_df.columns)
original_features.remove('Id')
original_features = set(original_features)

engineered_features = set(X_train) - original_features

top_mean = features_var['mean'].sort_values(ascending=False)[:10]
top_var = features_var['var'].sort_values(ascending=False)[:10]

top_mean_features = set(top_mean.index).intersection(engineered_features)
top_var_features = set(top_var.index).intersection(engineered_features)

print('original_features: {}'.format(len(original_features)))
print('top_mean_features: {}'.format(len(top_mean_features)))
print('top_var_features: {}'.format(len(top_var_features)))

print('top_mean_features + top_var_features: {}'.format(len(
                                                            top_mean_features.union(top_var_features)
                                                           
                                                           )))


selected_top_features = original_features.union(
                                                top_mean_features.union(top_var_features)
                                               )

print('Total Features: {}'.format(len(selected_top_features)))
print()
print(top_mean_features)
print()
print(top_var_features)
print()
print(top_mean_features.union(top_var_features))
print()

original_features: 12
top_mean_features: 9
top_var_features: 10
top_mean_features + top_var_features: 10
Total Features: 22

{'MEAN_3_COLS(Aspect, Elevation, Horizontal_Distance_To_Roadways)', 'MEAN_3_COLS(Elevation, Hillshade_3pm, Horizontal_Distance_To_Roadways)', 'MEAN_3_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways, Vertical_Distance_To_Hydrology)', 'MEAN_3_COLS(Hillshade_9am, Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)', 'MEAN_3_COLS(Elevation, Horizontal_Distance_To_Hydrology, Horizontal_Distance_To_Roadways)', 'MEAN_3_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Hydrology, Horizontal_Distance_To_Roadways)', 'MEAN_3_COLS(Elevation, Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)', 'MEAN_3_COLS(Hillshade_3pm, Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)', 'MEAN_3_COLS(Aspect, Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)'}

{'MEAN_3_COLS(Aspect

In [36]:
display(pd.DataFrame(top_mean_features))
display(pd.DataFrame(top_var_features))

,0
0,"MEAN_3_COLS(Aspect, Elevation, Horizontal_Dist..."
1,"MEAN_3_COLS(Elevation, Hillshade_3pm, Horizont..."
2,MEAN_3_COLS(Horizontal_Distance_To_Fire_Points...
3,"MEAN_3_COLS(Hillshade_9am, Horizontal_Distance..."
4,"MEAN_3_COLS(Elevation, Horizontal_Distance_To_..."
5,MEAN_3_COLS(Horizontal_Distance_To_Fire_Points...
6,"MEAN_3_COLS(Elevation, Horizontal_Distance_To_..."
7,"MEAN_3_COLS(Hillshade_3pm, Horizontal_Distance..."
8,"MEAN_3_COLS(Aspect, Horizontal_Distance_To_Fir..."


,0
0,"MEAN_3_COLS(Aspect, Elevation, Horizontal_Dist..."
1,"MEAN_3_COLS(Elevation, Hillshade_3pm, Horizont..."
2,MEAN_3_COLS(Horizontal_Distance_To_Fire_Points...
3,"MEAN_3_COLS(Elevation, Hillshade_9am, Horizont..."
4,"MEAN_3_COLS(Hillshade_9am, Horizontal_Distance..."
5,"MEAN_3_COLS(Elevation, Horizontal_Distance_To_..."
6,MEAN_3_COLS(Horizontal_Distance_To_Fire_Points...
7,"MEAN_3_COLS(Elevation, Horizontal_Distance_To_..."
8,"MEAN_3_COLS(Hillshade_3pm, Horizontal_Distance..."
9,"MEAN_3_COLS(Aspect, Horizontal_Distance_To_Fir..."


# Re-Train using selected features

In [37]:
fit_pipelines(pipelines, 
                X_train=X_train[selected_top_features], y_train=y_train,
                 X_test=X_test[selected_top_features], y_test=y_test)

Start fitting using Random Forest
Random Forest       : score=0.7714281170258636
Start fitting using Extra Trees
Extra Trees         : score=0.7913771532377203
Start fitting using KNN
KNN                 : score=0.6829660076481024
Start fitting using LGBM
LGBM                : score=0.7619598792702494



{'Random Forest': 0.7714281170258636,
 'Extra Trees': 0.7913771532377203,
 'KNN': 0.6829660076481024,
 'LGBM': 0.7619598792702494}

# Save Features

In [38]:
feature_objs = []

all_selected_features = list(top_mean_features.union(top_var_features))
for f in features:
    if f.get_name() in all_selected_features:
        feature_objs.append(f)

In [39]:
len(all_selected_features)

10

In [40]:
ft.save_features(feature_objs,
                location='./features.json')

### Calculating Feature Matrix for New Data

In [41]:
saved_features = ft.load_features('./features.json')

#### calculate the feature matrix.

In [42]:
%%time
feature_matrix_restored = ft.calculate_feature_matrix(saved_features, es)

CPU times: user 4.55 s, sys: 360 ms, total: 4.91 s
Wall time: 4.39 s


In [43]:
feature_matrix_restored.shape

(581012, 10)

In [44]:
feature_matrix_restored.head()

,"MEAN_3_COLS(Aspect, Elevation, Horizontal_Distance_To_Roadways)","MEAN_3_COLS(Hillshade_3pm, Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)","MEAN_3_COLS(Hillshade_9am, Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)","MEAN_3_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Hydrology, Horizontal_Distance_To_Roadways)","MEAN_3_COLS(Aspect, Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)","MEAN_3_COLS(Elevation, Hillshade_3pm, Horizontal_Distance_To_Roadways)","MEAN_3_COLS(Elevation, Hillshade_9am, Horizontal_Distance_To_Roadways)","MEAN_3_COLS(Elevation, Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)","MEAN_3_COLS(Elevation, Horizontal_Distance_To_Hydrology, Horizontal_Distance_To_Roadways)","MEAN_3_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways, Vertical_Distance_To_Hydrology)"
Id,,,,,,,,,,
1,1052.333333,2312.333333,2336.666667,2349.000000,2280.000000,1084.666667,1109.000000,3128.333333,1121.333333,2263.000000
2,1012.000000,2255.333333,2278.333333,2275.666667,2223.666667,1043.666667,1066.666667,3068.333333,1064.000000,2203.000000
3,2041.000000,3145.333333,3178.333333,3189.666667,3146.666667,2039.666667,2072.666667,4035.000000,2084.000000,3122.000000
4,2010.000000,3141.000000,3179.666667,3181.000000,3152.000000,1999.000000,2037.666667,4028.666667,2039.000000,3139.666667
5,1010.333333,2237.666667,2261.000000,2238.666667,2202.666667,1045.333333,1068.666667,3052.666667,1046.333333,2187.333333


# Stacking

In [45]:
# classifiers = []

# for name, p in pipelines.items():
#     if 'KNN'.lower() in name.lower():
#         continue
        
#     classifiers.append(p.steps[-1][1])

In [46]:
# %%time
# from mlxtend.classifier import EnsembleVoteClassifier

# eclf = EnsembleVoteClassifier(clfs=classifiers,
#                              voting='soft')


# score = fit_predict(eclf)

# print('score={}'.format(score))


In [47]:
# %%time
# from mlxtend.classifier import EnsembleVoteClassifier

# eclf = EnsembleVoteClassifier(clfs=classifiers,
#                              voting='hard')


# score = fit_predict(eclf)

# print('score={}'.format(score))
